
#            TP Identification de paramètres d'un robot RRR par techniques des moindres carrées #



Soit le robot RRR correspondant à la figure suivante:

![alternative text](robot-rrr3-sol.png)

- La configuration du robot est défini par le vecteur $q = (q_1, q_2, q_3)^t$.

- La situation de l'outil est défini par le vecteur $X = (x, y, z)^t$ .

## Objectif
L'objectif est d'identifier certains paramètres géométriques du robot plan RRR.

A l'aide d'une caméra, on a mesuré le vecteur $X$ pour un ensemble de configurations $q$. On considère que toutes les mesures sont exactes (pas d'erreurs liées au système de mesure).

Les résultats se trouvent dans les fichier *mesuresX.dat*. Chaque ligne comporte une mesure avec la convention suivante pour les 6 valeurs:

$q_1$, $q_2$, $q_3$, x, y, z sur chaque ligne (format %12.6f pour chaque valeur). Les longueurs sont exprimés en mètre.

- A: Dans un premier temps on veut identifier les longueurs des liaisons $a$, $b$, $c$ du robot RRR en considérant que les valeurs des angles $q_i$ sont exactes.

Fichier de mesure: *mesures1.dat*


## Travail

Modéliser le problème comme un problème d'optimisation

Implémenter votre problème et le résoudre 


In [7]:
#################################################   
# Import 
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import scipy
import time
from random import *
from numpy.linalg import inv,cond
from scipy.optimize import root
from scipy.optimize import least_squares

In [9]:
#################################################   
# Calcul du MGD du robot RRR
# INPUT:  q = vecteur de configuration (deg, deg, deg)
# OUTPUT: Xc = vecteur de situation = position3D = (x,y,z) 
def mgd(qdeg,a,b,c):
    qrad=np.deg2rad(qdeg)
    # introduction d'un eerreur su run qi
    # erreur= 5 (en degré)
    # erg = np.deg2rad(erreur)
    # qrad[i] = qi + erg
    c1= np.cos(qrad[0])
    s1=np.sin(qrad[0])
    c23= np.cos(qrad[2]+qrad[1])
    s23= np.sin(qrad[2]+qrad[1])
    c2=np.cos(qrad[1])
    s2=np.sin(qrad[1])

    A=[ [c1*c2, c1*c23, 0],
        [s1*c2, s2*c23, 0],
        [s2   , s23   , 1]]
    x= a*c1*c2 + b*c1*c23 
    y= a*s1*c2 + b*s1*c23 
    z= a*s2 + b*s23 + c 
    Xd=[x,y,z]
    return (Xd,A)

#

In [10]:
# Lecture mesures de fichier

with open("mesures1.dat", "r") as f:
    line= list()

    donnees= list()
    xdata= list()
    qdata= list()
    q1data= list()
    q2data= list()
    q3data= list()
    for line in f:
        if "#" in line:
            # on saute la ligne
            continue
        data = line.split()
        donnees.append((float(data[0]), float(data[1]),float(data[2]), float(data[3]), float(data[4]),float(data[5])))
        xdata.append([float(data[3]), float(data[4]),float(data[5])])
        qdata.append([float(data[0]), float(data[1]),float(data[2])])
        q1data.append(float(data[0]))
        q2data.append(float(data[1]))
        q3data.append(float(data[2]))
#donnees
#xdata


In [17]:


a,b,c=1,1,1
r=[]
A= []
b=[]
X = [1,1,1]
for i in range(0,len(qdata)):
    while np.linalg.norm(-mgd(qdata[i],X[0],X[1],X[2])[0] + xdata[i]) > 1 :
        A= np.array(mgd(qdata[i],X[0],X[1],X[2])[1])
        b = np.array(xdata[i])
        print(A)
        print(b)
        X = np.dot(np.linalg.inv(A.transpose()*A) * A.transpose() , b)
        print(X)
        # A.append(-mgd(qdata[i],a,b,c))
        
        # r.append(-mgd(qdata[i],a,b,c) + xdata[i])


print(X)
# Calcul de a, b et c

# print("a=", "b=", ,"c=", )

[[ 0.25       0.25       0.       ]
 [ 0.4330127  0.4330127  0.       ]
 [-0.8660254  0.8660254  1.       ]]
[1.652013 2.797038 1.955745]
[-6.99010234e+16  4.03573747e+16  1.95574500e+00]
[[ 0.25       0.4330127  0.       ]
 [ 0.4330127  0.75       0.       ]
 [-0.8660254  0.5        1.       ]]
[2.007306 3.556707 1.074906]
[ 5.25183171e+14 -3.03214645e+14  1.07490600e+00]
[[ 2.50000000e-01  5.00000000e-01  0.00000000e+00]
 [ 4.33012702e-01  8.66025404e-01  0.00000000e+00]
 [-8.66025404e-01 -1.22464680e-16  1.00000000e+00]]
[ 2.291994  3.882584 -0.100809]
[-8.13391571e+15  4.69611842e+15 -1.00809000e-01]
[[ 0.25       0.4330127  0.       ]
 [ 0.4330127  0.75       0.       ]
 [-0.8660254 -0.5        1.       ]]
[ 2.159217  3.650892 -1.497598]
[-1.04114597e+15  6.01105903e+14 -1.49759800e+00]
[[ 0.25       0.25       0.       ]
 [ 0.4330127  0.4330127  0.       ]
 [-0.8660254 -0.8660254  1.       ]]
[ 1.684815  2.754952 -2.306094]
[-2.41748552e+16  1.39573591e+16 -2.30609400e+00]
[[ 2.5

LinAlgError: Singular matrix

### Test du MGD après identification

Vérifier que votre identification des longueurs $a$, $b$ et $c$ est correcte.

## Utilisation de scipy optimize

Retrouver vos résultats en utilisant directement les fonctions de scipy optimize